In [2]:
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import model_selection

from sklearn.datasets import fetch_20newsgroups
from keras.layers import  Dropout, Dense
from keras.models import Sequential

##Importamos las matrices a utilizar

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1Pk5MK9Hs_kMUT9NotGnOKE0NPra-39YU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train.csv')

train = pd.read_csv('train.csv')

id='1GsTM9oLtIV8-Da_fDOFWsQYMpgQ8GOYJ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test.csv')

test = pd.read_csv('test.csv')

In [5]:
id= '1OrAkBep-bmOz-P-QR3P5csDejgiJG9bL'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('doc2vec_features_numericos_test.csv')

id= '17bNeSA9q59vTO6OW2lBTJPyw29QGQSxf'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('doc2vec_features_numericos_train.csv')

id= '1Ffgm1RoYdjqMuQgTap3o5bOSaNjIdO7l'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('matriz_test_doc2vec_solo_text.csv')

id= '1Q7m92Pbm08m0t9X8Q975IUkPWEoZQ--R'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('matriz_train_doc2vec_solo_text.csv')

id= '1qvK0vblQPKJSgE8dTphy7sactLSVp6xf'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('matriz_test_doc2vec_text_keyword.csv')

id= '1bQDT6HcEy46m2j2n92OLi5NmufCqaZ5Z'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('matriz_train_doc2vec_text_keyword.csv')

In [6]:
matriz_test_doc2vec_solo_text = pd.read_csv('matriz_test_doc2vec_solo_text.csv')
matriz_train_doc2vec_solo_text = pd.read_csv('matriz_train_doc2vec_solo_text.csv')

matriz_test_doc2vec_text_keyword = pd.read_csv('matriz_test_doc2vec_text_keyword.csv')
matriz_train_doc2vec_text_keyword = pd.read_csv('matriz_train_doc2vec_text_keyword.csv')

doc2vec_features_numericos_test = pd.read_csv('doc2vec_features_numericos_test.csv')
doc2vec_features_numericos_train = pd.read_csv('doc2vec_features_numericos_train.csv')

In [19]:
def cross_val(model, x_train, y_train):
  score_cross_val = model_selection.cross_val_score(model, x_train, y_train, cv=5)
  print(score_cross_val.mean())

##SPLIT DE LAS MATRICES

In [7]:
y = train.target

In [8]:
x_train_text, x_validation_text, y_train_text, y_validation_text = train_test_split(matriz_train_doc2vec_solo_text, y, test_size=0.3, stratify=y)
x_train_text_keyword, x_validation_text_keyword, y_train_text_keyword, y_validation_text_keyword = train_test_split(matriz_train_doc2vec_text_keyword, y, test_size=0.3, stratify=y)
x_train_numericos, x_validation_numericos, y_train_numericos, y_validation_numericos = train_test_split(doc2vec_features_numericos_train, y, test_size=0.3, stratify=y)

##KNN

Funcion que aplica KNN con GridSearch y CV para encontrar los mejores hiperparametros para cada matriz probada.

In [9]:
def knn_con_gridsearch(x_train, y_train, x_validation, y_validation):

  knn = KNeighborsClassifier()
  params_knn = {'n_neighbors': np.arange(1, 25)}
  knn_gs = GridSearchCV(knn, params_knn, cv=5)
  knn_gs.fit(x_train, y_train)
  knn_best = knn_gs.best_estimator_
  
  print(knn_gs.best_params_)
  print('knn score: {}'.format(knn_best.score(x_validation, y_validation)))
  
  return knn_best

Corremos el algoritmo con cada matriz y comparamos los resultados.

In [20]:
knn_text = knn_con_gridsearch(x_train_text, y_train_text, x_validation_text, y_validation_text)

{'n_neighbors': 20}
knn score: 0.5652364273204904


In [21]:
cross_val(knn_text, x_train_text, y_train_text)

0.5697118797840199


In [27]:
knn_text_keyword = knn_con_gridsearch(x_train_text_keyword, y_train_text_keyword, x_validation_text_keyword, y_validation_text_keyword)

{'n_neighbors': 10}
knn score: 0.6112084063047285


In [28]:
cross_val(knn_text_keyword, x_train_text_keyword, y_train_text_keyword)

0.5978625725585534


In [29]:
knn_numericos = knn_con_gridsearch(x_train_numericos,y_train_numericos,  x_validation_numericos, y_validation_numericos)

{'n_neighbors': 24}
knn score: 0.658493870402802


In [30]:
cross_val(knn_numericos, x_train_text, y_train_text)

0.5627661654731391


Realizamos la prediccion con el modelo que haya dado mayor score

In [ ]:
#prediction_knn = knn_best.predict(doc2vec_features_numericos_test)
#submission_knn = pd.DataFrame(data={'id':test['id'], 'target': prediction_knn})
#submission_knn.to_csv('doc2vec_knn.csv', index=False)

#RED NEURONAL PROFUNDA

In [31]:
def DNN_model(shape, nClasses, dropout=0.5):
    model = Sequential()
    node = 512
    nLayers = 4
    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Corremos el algoritmo con cada matriz y comparamos los resultados.

In [32]:
model_DNN_text = DNN_model(matriz_train_doc2vec_solo_text.shape[1], 2)
model_DNN_text.fit(matriz_train_doc2vec_solo_text, train.target, epochs=20, batch_size=128, verbose=2)

Epoch 1/20
60/60 - 2s - loss: 0.6965 - accuracy: 0.5479
Epoch 2/20
60/60 - 2s - loss: 0.6860 - accuracy: 0.5627
Epoch 3/20
60/60 - 2s - loss: 0.6819 - accuracy: 0.5702
Epoch 4/20
60/60 - 2s - loss: 0.6817 - accuracy: 0.5672
Epoch 5/20
60/60 - 2s - loss: 0.6790 - accuracy: 0.5698
Epoch 6/20
60/60 - 2s - loss: 0.6764 - accuracy: 0.5812
Epoch 7/20
60/60 - 2s - loss: 0.6731 - accuracy: 0.5799
Epoch 8/20
60/60 - 2s - loss: 0.6704 - accuracy: 0.5801
Epoch 9/20
60/60 - 2s - loss: 0.6646 - accuracy: 0.5939
Epoch 10/20
60/60 - 2s - loss: 0.6576 - accuracy: 0.6042
Epoch 11/20
60/60 - 2s - loss: 0.6499 - accuracy: 0.6137
Epoch 12/20
60/60 - 2s - loss: 0.6425 - accuracy: 0.6226
Epoch 13/20
60/60 - 2s - loss: 0.6380 - accuracy: 0.6291
Epoch 14/20
60/60 - 2s - loss: 0.6269 - accuracy: 0.6495
Epoch 15/20
60/60 - 2s - loss: 0.6147 - accuracy: 0.6548
Epoch 16/20
60/60 - 2s - loss: 0.6053 - accuracy: 0.6707
Epoch 17/20
60/60 - 2s - loss: 0.6002 - accuracy: 0.6685
Epoch 18/20
60/60 - 2s - loss: 0.5876 - 

In [33]:
model_DNN_text_keyword = DNN_model(matriz_train_doc2vec_text_keyword.shape[1], 2)
model_DNN_text_keyword.fit(matriz_train_doc2vec_text_keyword, train.target, epochs=20, batch_size=128, verbose=2)

Epoch 1/20
60/60 - 2s - loss: 0.6945 - accuracy: 0.5462
Epoch 2/20
60/60 - 2s - loss: 0.6824 - accuracy: 0.5672
Epoch 3/20
60/60 - 2s - loss: 0.6767 - accuracy: 0.5738
Epoch 4/20
60/60 - 2s - loss: 0.6724 - accuracy: 0.5784
Epoch 5/20
60/60 - 2s - loss: 0.6698 - accuracy: 0.5835
Epoch 6/20
60/60 - 2s - loss: 0.6615 - accuracy: 0.5940
Epoch 7/20
60/60 - 2s - loss: 0.6568 - accuracy: 0.6099
Epoch 8/20
60/60 - 2s - loss: 0.6474 - accuracy: 0.6226
Epoch 9/20
60/60 - 2s - loss: 0.6413 - accuracy: 0.6386
Epoch 10/20
60/60 - 2s - loss: 0.6243 - accuracy: 0.6440
Epoch 11/20
60/60 - 2s - loss: 0.6154 - accuracy: 0.6643
Epoch 12/20
60/60 - 2s - loss: 0.6089 - accuracy: 0.6653
Epoch 13/20
60/60 - 2s - loss: 0.5966 - accuracy: 0.6824
Epoch 14/20
60/60 - 2s - loss: 0.5856 - accuracy: 0.6932
Epoch 15/20
60/60 - 2s - loss: 0.5756 - accuracy: 0.6991
Epoch 16/20
60/60 - 2s - loss: 0.5614 - accuracy: 0.7119
Epoch 17/20
60/60 - 2s - loss: 0.5508 - accuracy: 0.7167
Epoch 18/20
60/60 - 2s - loss: 0.5398 - 

In [34]:
model_DNN_text_keyword_numericos = DNN_model(doc2vec_features_numericos_train.shape[1], 2)
model_DNN_text_keyword_numericos.fit(doc2vec_features_numericos_train, train.target, epochs=20, batch_size=128, verbose=2)

Epoch 1/20
60/60 - 2s - loss: 0.9790 - accuracy: 0.5307
Epoch 2/20
60/60 - 2s - loss: 0.7077 - accuracy: 0.5392
Epoch 3/20
60/60 - 2s - loss: 0.6897 - accuracy: 0.5621
Epoch 4/20
60/60 - 2s - loss: 0.6791 - accuracy: 0.5667
Epoch 5/20
60/60 - 2s - loss: 0.6677 - accuracy: 0.5854
Epoch 6/20
60/60 - 2s - loss: 0.6638 - accuracy: 0.5986
Epoch 7/20
60/60 - 2s - loss: 0.6526 - accuracy: 0.6113
Epoch 8/20
60/60 - 2s - loss: 0.6527 - accuracy: 0.6187
Epoch 9/20
60/60 - 2s - loss: 0.6442 - accuracy: 0.6327
Epoch 10/20
60/60 - 2s - loss: 0.6375 - accuracy: 0.6419
Epoch 11/20
60/60 - 2s - loss: 0.6358 - accuracy: 0.6390
Epoch 12/20
60/60 - 2s - loss: 0.6338 - accuracy: 0.6471
Epoch 13/20
60/60 - 2s - loss: 0.6323 - accuracy: 0.6524
Epoch 14/20
60/60 - 2s - loss: 0.6306 - accuracy: 0.6447
Epoch 15/20
60/60 - 2s - loss: 0.6259 - accuracy: 0.6649
Epoch 16/20
60/60 - 2s - loss: 0.6286 - accuracy: 0.6522
Epoch 17/20
60/60 - 2s - loss: 0.6228 - accuracy: 0.6622
Epoch 18/20
60/60 - 2s - loss: 0.6245 - 

In [ ]:
#prediction_dnn = model_DNN_text_keyword.predict_classes(matriz_test_doc2vec_text_keyword)
#submission_dnn = pd.DataFrame(data={'id':test['id'], 'target': prediction_dnn})
#submission_dnn.to_csv('doc2vec_dnn.csv', index=False)

#RANDOM FOREST

Funcion que aplica RANDOM FOREST con GridSearch y CV para encontrar los mejores hiperparametros para cada matriz probada.

In [36]:
def RandomForest_con_gridsearch(x_train, y_train, x_validation, y_validation):

  rf = RandomForestClassifier()
  params_rf = {'n_estimators': [50, 100, 200]}
  rf_gs = GridSearchCV(rf, params_rf, cv=5)
  rf_gs.fit(x_train, y_train)
  rf_best = rf_gs.best_estimator_

  print(rf_gs.best_params_)
  print('rf: {}'.format(rf_best.score(x_validation, y_validation)))

  return rf_best

Corremos el algoritmo con cada matriz y comparamos los resultados.

In [37]:
rf_text = RandomForest_con_gridsearch(x_train_text, y_train_text, x_validation_text, y_validation_text)

{'n_estimators': 200}
rf: 0.5595446584938704


In [38]:
cross_val(rf_text, x_train_text, y_train_text)

0.5704623488271718


In [39]:
rf_text_keyword = RandomForest_con_gridsearch(x_train_text_keyword, y_train_text_keyword, x_validation_text_keyword, y_validation_text_keyword)

{'n_estimators': 200}
rf: 0.6265323992994746


In [40]:
cross_val(rf_text_keyword, x_train_text_keyword, y_train_text_keyword)

0.5926064705934166


In [41]:
rf_numericos = RandomForest_con_gridsearch(x_train_numericos,y_train_numericos,  x_validation_numericos, y_validation_numericos)

{'n_estimators': 200}
rf: 0.6580560420315237


In [43]:
cross_val(rf_numericos, x_train_numericos, y_train_numericos)

0.6701079019457583


In [ ]:
#prediction_rf_text = rf_text.predict(doc2vec_features_numericos_test)
#submission_rf = pd.DataFrame(data={'id':test['id'], 'target': prediction_rf})
#submission_rf.to_csv('doc2vec_rf.csv', index=False)

#NuSVC

In [49]:
def nusvc(x_train, y_train):

  nusvc = NuSVC(probability=True)
  grid_nusvc = GridSearchCV(nusvc, param_grid={'nu': [0.4, 0.5]}, scoring=['f1_macro'], refit=False)
  grid_nusvc.fit(x_train, y_train)

  print('nusvc: {}'.format(grid_nusvc.cv_results_['mean_test_f1_macro'][0]))

  return grid_nusvc

In [50]:
nusvc_text = nusvc(x_train_text, y_train_text)

nusvc: 0.5399388147847999


In [52]:
nusvc_text_keyword = nusvc(x_train_text_keyword, y_train_text_keyword)

nusvc: 0.5843175548937947


In [53]:
nusvc_numericos = nusvc(x_train_numericos,y_train_numericos)

nusvc: 0.6185243425699021


In [ ]:
#prediction_nuscv = nuscv.predict(doc2vec_features_numericos_test)
#submission_nuscv = pd.DataFrame(data={'id':test['id'], 'target': prediction_nuscv})
#submission_nuscv.to_csv('doc2vec_nuscv.csv', index=False)

#XGBoost

In [54]:
def xgboost(x_train, y_train, x_validation, y_validation):

  model = XGBClassifier()
  model.fit(x_train, y_train)
  y_pred = model.predict(x_validation)
  predictions = [round(value) for value in y_pred]
  accuracy = accuracy_score(y_validation, predictions)

  print("Accuracy xgboost: %.5f%%" % accuracy)

  return model

In [55]:
xgboost_text = xgboost(x_train_text, y_train_text, x_validation_text, y_validation_text)

Accuracy xgboost: 0.55298%


In [56]:
cross_val(xgboost_text, x_train_text, y_train_text)

0.5661482088277004


In [57]:
xgboost_text_keyword = xgboost(x_train_text_keyword, y_train_text_keyword, x_validation_text_keyword, y_validation_text_keyword)

Accuracy xgboost: 0.60114%


In [58]:
cross_val(xgboost_text_keyword, x_train_text_keyword, y_train_text_keyword)

0.587725603149856


In [59]:
xgboost_numericos = xgboost(x_train_numericos,y_train_numericos,  x_validation_numericos, y_validation_numericos)

Accuracy xgboost: 0.66243%


In [60]:
cross_val(xgboost_numericos, x_train_numericos,y_train_numericos)

0.6734828986426374


In [ ]:
#submission_xgboost = pd.DataFrame(data={'id':test['id'], 'target': prediction_xgboost})
#submission_xgboost.to_csv('tf_idf_xgboost.csv', index=False)